## 🔗 Useful Links

- [Riot Developer Portal](https://developer.riotgames.com/)  
  👉 Used to generate and manage your **Riot API Key** (required for all API calls).

- [OP.GG Leaderboards (EUW Example)](https://op.gg/lol/leaderboards/tier?region=euw)  
  👉 Can be used to manually check or test players from the leaderboard if you want to try specific cases.


In [ ]:
# === 0) Libraries
import requests, time, pandas as pd

# Make pandas show all columns in Colab outputs
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

# === 1) Config (EDIT THESE)
API_KEY   = "RGAPI-e5a65005-3c43-4880-8aa6-912a1e594df9"   # Get from https://developer.riotgames.com/
RIOT_ID   = "dmsdklb"                   # e.g., "Faker"
TAGLINE   = "vivi"                          # e.g., "EUW", "NA1", "KR1"
PLATFORM  = "euw1"                           # Platform routing: e.g., "euw1", "na1", "kr", "br1", ...
# Platform examples: EUW -> "euw1", NA -> "na1", KR -> "kr", EUNE -> "eun1", BR -> "br1", JP -> "jp1", LAN -> "la1", LAS -> "la2", OCE -> "oc1", TR -> "tr1", RU -> "ru"

# Regional routing depends on PLATFORM (for Account and Match APIs)
def platform_to_region(platform: str) -> str:
    platform = platform.lower()
    if platform in {"na1", "br1", "la1", "la2", "oc1"}:
        return "americas"
    if platform in {"euw1", "eun1", "tr1", "ru"}:
        return "europe"
    if platform in {"kr", "jp1"}:
        return "asia"
    # Fallback (adjust if needed)
    return "americas"

REGION = platform_to_region(PLATFORM)

# === 2) Session with headers + simple rate-limit helper
session = requests.Session()
session.headers.update({"X-Riot-Token": API_KEY})

def riot_get(url, params=None, retry=3, backoff=1.2):
    """Tiny helper to GET with basic retry (429/5xx)."""
    for i in range(retry):
        r = session.get(url, params=params, timeout=30)
        if r.status_code == 200:
            return r.json()
        if r.status_code in (429, 500, 502, 503, 504):
            time.sleep((i+1)*backoff)
            continue
        # Raise for other errors (401, 403, 404, etc.)
        r.raise_for_status()
    # Last try
    r = session.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

# === 3) Look up player (PUUID) from Riot ID + Tagline
# Account-V1 uses REGIONAL routing (americas/europe/asia)
acc_url = f"https://{REGION}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{RIOT_ID}/{TAGLINE}"
account = riot_get(acc_url)
puuid   = account["puuid"]

# (Optional) Summoner core info (platform routing)
summoner_url = f"https://{PLATFORM}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
summoner = riot_get(summoner_url)

print(f"PUUID: {puuid}")
print(f"Summoner name: {summoner.get('name')} | Level: {summoner.get('summonerLevel')}")

# === 4) Get recent match IDs, then pull each match (Match-V5 uses REGIONAL routing)
# You can increase 'count' if you want more matches (max usually 100 per call)
matchlist_url = f"https://{REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
match_ids = riot_get(matchlist_url, params={"start": 0, "count": 20})
print(f"Fetched {len(match_ids)} match IDs.")

# Pull full match JSONs
matches_raw = []
for mid in match_ids:
    m_url = f"https://{REGION}.api.riotgames.com/lol/match/v5/matches/{mid}"
    matches_raw.append(riot_get(m_url))

# === 5) Show ALL columns (no dropping) — two views:
# (A) Entire match objects (metadata + info.*) flattened
matches_df = pd.json_normalize(matches_raw, sep=".")
print(f"\n[ALL MATCH FIELDS] shape={matches_df.shape} (rows, cols)")
display(matches_df.head(3))
print("ALL match columns:\n", list(matches_df.columns))

# (B) Only the participant rows that correspond to THIS player across matches (still all fields)
#    This is often what you want to analyze for the specific player (KDA, vision, runes, damage, items, etc.)
player_rows = []
for m in matches_raw:
    parts = m.get("info", {}).get("participants", [])
    for p in parts:
        if p.get("puuid") == puuid:
            # include metadata & match info keys at top level too (like gameId) for context
            base = {
                "metadata.matchId": m.get("metadata", {}).get("matchId"),
                "info.gameCreation": m.get("info", {}).get("gameCreation"),
                "info.gameDuration": m.get("info", {}).get("gameDuration"),
                "info.gameMode":     m.get("info", {}).get("gameMode"),
                "info.gameVersion":  m.get("info", {}).get("gameVersion"),
            }
            # Merge the entire participant dict (ALL COLUMNS)
            base.update(p)
            player_rows.append(base)

player_df = pd.json_normalize(player_rows, sep=".")
print(f"\n[PLAYER-ONLY ROWS] shape={player_df.shape} (rows, cols)")
display(player_df.head(3))
print("ALL player columns:\n", list(player_df.columns))

# === 6) (Optional) Save everything for offline inspection in Excel/CSV
#matches_df.to_csv("all_match_fields.csv", index=False)
player_df.to_csv("player_rows_fields.csv", index=False)

# Tip: In Colab, you can scroll sideways in the table, or open the saved CSV to inspect all columns.


PUUID: 5Iw-9waAaKjvGDeoQ5etuOWhd3CS8oxEIIpCwRnbb_HaXQRvSeO7G8N0i12h1s5OFD6ppd11aZqvRA
Summoner name: None | Level: 104
Fetched 20 match IDs.

[ALL MATCH FIELDS] shape=(20, 19) (rows, cols)


,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.gameCreation,info.gameDuration,info.gameEndTimestamp,info.gameId,info.gameMode,info.gameName,info.gameStartTimestamp,info.gameType,info.gameVersion,info.mapId,info.participants,info.platformId,info.queueId,info.teams,info.tournamentCode
0,2,EUW1_7516269315,[DLMDKXhNPu0cRx7h0TwXcrtFD84rHUzzrvIzAK2rXHhOg...,GameComplete,1756820882724,1702,1756822604281,7516269315,CLASSIC,teambuilder-match-7516269315,1756820901862,MATCHED_GAME,15.17.708.5788,11,"[{'PlayerScore0': 0, 'PlayerScore1': 0, 'Playe...",EUW1,420,"[{'bans': [{'championId': 107, 'pickTurn': 1},...",
1,2,EUW1_7516229163,[MC692oIt_fw5NhP-un0JeH2x0aESXqpXj55HVRHv2zqOo...,GameComplete,1756817912708,1866,1756819805996,7516229163,CLASSIC,teambuilder-match-7516229163,1756817939431,MATCHED_GAME,15.17.708.5788,11,"[{'PlayerScore0': 0, 'PlayerScore1': 0, 'Playe...",EUW1,420,"[{'bans': [{'championId': 91, 'pickTurn': 1}, ...",
2,2,EUW1_7515921608,[fwpQGi5MCO2LvXjPwcc6KIpVZNGGwtt0LlzETyi_z4QKo...,GameComplete,1756767828021,2045,1756769890437,7515921608,CLASSIC,teambuilder-match-7515921608,1756767845605,MATCHED_GAME,15.17.708.5788,11,"[{'PlayerScore0': 0, 'PlayerScore1': 0, 'Playe...",EUW1,420,"[{'bans': [{'championId': 119, 'pickTurn': 1},...",


ALL match columns:
 ['metadata.dataVersion', 'metadata.matchId', 'metadata.participants', 'info.endOfGameResult', 'info.gameCreation', 'info.gameDuration', 'info.gameEndTimestamp', 'info.gameId', 'info.gameMode', 'info.gameName', 'info.gameStartTimestamp', 'info.gameType', 'info.gameVersion', 'info.mapId', 'info.participants', 'info.platformId', 'info.queueId', 'info.teams', 'info.tournamentCode']

[PLAYER-ONLY ROWS] shape=(20, 300) (rows, cols)


,metadata.matchId,info.gameCreation,info.gameDuration,info.gameMode,info.gameVersion,PlayerScore0,PlayerScore1,PlayerScore10,PlayerScore11,PlayerScore2,PlayerScore3,PlayerScore4,PlayerScore5,PlayerScore6,PlayerScore7,PlayerScore8,PlayerScore9,allInPings,assistMePings,assists,baronKills,basicPings,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,placement,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerAugment5,playerAugment6,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,retreatPings,riotIdGameName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.HealFromMapSources,challenges.InfernalScalePickup,challenges.SWARM_DefeatAatrox,challenges.SWARM_DefeatBriar,challenges.SWARM_DefeatMiniBosses,challenges.SWARM_EvolveWeapon,challenges.SWARM_Have3Passives,challenges.SWARM_KillEnemy,challenges.SWARM_PickupGold,challenges.SWARM_ReachLevel50,challenges.SWARM_Survive15Min,challenges.SWARM_WinWith5EvolvedWeapons,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronBuffGoldAdvantageOverThreshold,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardTimeCoverageInRiverOrEnemyHalf,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.dancedWithRiftHerald,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earliestBaron,challenges.earliestDragonTakedown,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.fistBumpParticipation,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.highestChampionDamage,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCoun

ALL player columns:
 ['metadata.matchId', 'info.gameCreation', 'info.gameDuration', 'info.gameMode', 'info.gameVersion', 'PlayerScore0', 'PlayerScore1', 'PlayerScore10', 'PlayerScore11', 'PlayerScore2', 'PlayerScore3', 'PlayerScore4', 'PlayerScore5', 'PlayerScore6', 'PlayerScore7', 'PlayerScore8', 'PlayerScore9', 'allInPings', 'assistMePings', 'assists', 'baronKills', 'basicPings', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'commandPings', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'dangerPings', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'enemyMissingPings', 'enemyVisionPings', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'getBackPings', 'goldEarned', 'goldSpent', 'holdPings', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns',